In [1]:
import numpy as np
from fda import *
import pickle
from federatedAlgs import *
from dataGenerator import *
import time
# dataGenerator()

2024-01-16 02:06:21,582	INFO worker.py:1724 -- Started a local Ray instance.


In [2]:
# number of basis functions in basis system
t = 20
rangeval = [0, 100]
basisobj = create_bspline_basis(rangeval, t)
# number of observations pre local server n
n = 100
# number of features p 20
p = 20
betaPar = fdPar(basisobj, 0, 0)
# number of observations pre local server n
samplesPerWorker = 100
# number of features p 20
numPredictors = 20
yfine = np.linspace(1, 100, 100)
allsamples = set([i for i in range(100)])
numworkersseq = [2, 4, 6, 8, 10]
bbspl2 = bifd(np.linspace(1, pow(t, 2), pow(t, 2)).reshape((t, t)), create_bspline_basis(rangeval, t),
              create_bspline_basis(rangeval, t))
bifdbasis = bifdPar(bbspl2, 0, 0, 0, 0)
betaList = [betaPar, bifdbasis]
time1 = np.zeros([len(numworkersseq), 4])
time2 = np.zeros([len(numworkersseq), 4])

In [3]:
# orginal time: Number of local servers : 2; 0 fold.
# 1 1705387323.2915132
# 2 1705387370.324656
# 3 1705387370.324656
# 4 1705387373.236033
# LSA Round 2 : 14 selected.
# 5 1705387376.2423809
# 6 1705387376.2455325

# ray time:
# 1 1705388451.6095245
# 2 1705388456.0331485

In [4]:
for numworkers in numworkersseq:
    # 4-fold CV
    for hh in range(4):
        print(f'Number of local servers : {numworkers}; {hh} fold.')
        test = set([i for i in range(hh * 25, (hh + 1) * 25)])
        train = list(allsamples - test)
        test = setGenerator(test, samplesPerWorker, numworkers)
        predictors = np.zeros([len(yfine), samplesPerWorker * numworkers, numPredictors])
        response = np.zeros([len(yfine), samplesPerWorker * numworkers])
        x = []
        y = []
        for l in range(1, numworkers + 1):
            with open('tmp/yfdobj_' + str(l) + '_' + str(numworkers), 'rb') as file:
                yfdobj = pickle.load(file)
            with open('tmp/predictorLst_' + str(l) + '_' + str(numworkers), 'rb') as file:
                predictorLst = pickle.load(file)
            xfdobjTrainLst = []
            for i in range(numPredictors):
                predictors[:, (samplesPerWorker * (l - 1)): (samplesPerWorker * l), i] = eval_fd(yfine,
                                                                                                 predictorLst[i])
                temp1 = smooth_basis(yfine, predictors[:, [i + samplesPerWorker * (l - 1) - 1 for i in train], i],
                                     basisobj).fd
                xfdobjTrainLst.append(temp1)
            x.append(xfdobjTrainLst)
            response[:, (samplesPerWorker * (l - 1)): (samplesPerWorker * l)] = eval_fd(yfine, yfdobj)
            responsefdobjTrain = smooth_basis(yfine,
                                              response[:, [i + samplesPerWorker * (l - 1) - 1 for i in train]],
                                              basisobj).fd
            y.append(responsefdobjTrain)

        yfdobjTest = smooth_basis(yfine, response[:, test], basisobj).fd
        responseTest = eval_fd(yfine, yfdobjTest)
        boost_control = 10
        step_length = 0.5
        lin = linmod(x[0][1], y[0], betaList)
        step_length = 0.5
        start = time.time()
        federatedFunctionalGradBoostLSA(x, y, betaList, boost_control, step_length)
        end = time.time()
        time1[numworkersseq.index(numworkers), hh] = end - start
        start = time.time()
        federatedFunctionalGradBoostAvg(x, y, betaList, boost_control, step_length)
        end = time.time()
        time2[numworkersseq.index(numworkers), hh] = end - start
print("Comparison of operation times of fed-GB-LSA (LSA) and fed-GB-Average (Avg)")
print('operation times of fed-GB-LSA (LSA)')
print(np.mean(time1, 1))
print('operation times of fed-GB-Average (Avg)')
print(np.mean(time2, 1))

Number of local servers : 2; 0 fold.
LSA Round 2 : 9 selected.
LSA Round 3 : 9 selected.
LSA Round 4 : 8 selected.
LSA Round 5 : 9 selected.
LSA Round 6 : 9 selected.
LSA Round 7 : 11 selected.
LSA Round 8 : 8 selected.
LSA Round 9 : 8 selected.
LSA Round 10 : 2 selected.
AVG Round 2 : 0 selected.
AVG Round 3 : 0 selected.
AVG Round 4 : 0 selected.
AVG Round 5 : 0 selected.
AVG Round 6 : 0 selected.
AVG Round 7 : 1 selected.
AVG Round 8 : 0 selected.
AVG Round 9 : 0 selected.
AVG Round 10 : 1 selected.
Number of local servers : 2; 1 fold.
LSA Round 2 : 12 selected.
LSA Round 3 : 12 selected.
LSA Round 4 : 2 selected.
LSA Round 5 : 2 selected.
LSA Round 6 : 2 selected.
LSA Round 7 : 7 selected.
LSA Round 8 : 11 selected.
LSA Round 9 : 2 selected.
LSA Round 10 : 2 selected.
AVG Round 2 : 0 selected.
AVG Round 3 : 0 selected.
AVG Round 4 : 0 selected.
AVG Round 5 : 0 selected.
AVG Round 6 : 0 selected.
AVG Round 7 : 1 selected.
AVG Round 8 : 0 selected.
AVG Round 9 : 0 selected.
AVG Round

In [6]:
numworkersseq = [2, 4]
bbspl2 = bifd(np.linspace(1, pow(t, 2), pow(t, 2)).reshape((t, t)), create_bspline_basis(rangeval, t),
              create_bspline_basis(rangeval, t))
bifdbasis = bifdPar(bbspl2, 0, 0, 0, 0)
betaList = [betaPar, bifdbasis]
time1 = np.zeros([len(numworkersseq), 4])
time2 = np.zeros([len(numworkersseq), 4])

In [7]:

for numworkers in numworkersseq:
    # 4-fold CV
    for hh in range(4):
        print(f'Number of local servers : {numworkers}; {hh} fold.')
        test = set([i for i in range(hh * 25, (hh + 1) * 25)])
        train = list(allsamples - test)
        test = setGenerator(test, samplesPerWorker, numworkers)
        predictors = np.zeros([len(yfine), samplesPerWorker * numworkers, numPredictors])
        response = np.zeros([len(yfine), samplesPerWorker * numworkers])
        x = []
        y = []
        for l in range(1, numworkers + 1):
            with open('tmp/yfdobj_' + str(l) + '_' + str(numworkers), 'rb') as file:
                yfdobj = pickle.load(file)
            with open('tmp/predictorLst_' + str(l) + '_' + str(numworkers), 'rb') as file:
                predictorLst = pickle.load(file)
            xfdobjTrainLst = []
            for i in range(numPredictors):
                predictors[:, (samplesPerWorker * (l - 1)): (samplesPerWorker * l), i] = eval_fd(yfine,
                                                                                                 predictorLst[i])
                temp1 = smooth_basis(yfine, predictors[:, [i + samplesPerWorker * (l - 1) - 1 for i in train], i],
                                     basisobj).fd
                xfdobjTrainLst.append(temp1)
            x.append(xfdobjTrainLst)
            response[:, (samplesPerWorker * (l - 1)): (samplesPerWorker * l)] = eval_fd(yfine, yfdobj)
            responsefdobjTrain = smooth_basis(yfine,
                                              response[:, [i + samplesPerWorker * (l - 1) - 1 for i in train]],
                                              basisobj).fd
            y.append(responsefdobjTrain)

        yfdobjTest = smooth_basis(yfine, response[:, test], basisobj).fd
        responseTest = eval_fd(yfine, yfdobjTest)
        boost_control = 2
        step_length = 0.5
        lin = linmod(x[0][1], y[0], betaList)
        step_length = 0.5
        start = time.time()
        federatedFunctionalGradBoostLSA(x, y, betaList, boost_control, step_length)
        end = time.time()
        time1[numworkersseq.index(numworkers), hh] = end - start
        start = time.time()
        federatedFunctionalGradBoostAvg(x, y, betaList, boost_control, step_length)
        end = time.time()
        time2[numworkersseq.index(numworkers), hh] = end - start
print("Comparison of operation times of fed-GB-LSA (LSA) and fed-GB-Average (Avg)")
print('operation times of fed-GB-LSA (LSA)')
print(np.mean(time1, 1))
print('operation times of fed-GB-Average (Avg)')
print(np.mean(time2, 1))

Number of local servers : 2; 0 fold.
LSA Round 2 : 14 selected.
AVG Round 2 : 0 selected.
Number of local servers : 2; 1 fold.
LSA Round 2 : 0 selected.
AVG Round 2 : 0 selected.
Number of local servers : 2; 2 fold.
LSA Round 2 : 0 selected.
AVG Round 2 : 0 selected.
Number of local servers : 2; 3 fold.
LSA Round 2 : 0 selected.
AVG Round 2 : 0 selected.
Number of local servers : 4; 0 fold.
LSA Round 2 : 18 selected.
AVG Round 2 : 4 selected.
Number of local servers : 4; 1 fold.
LSA Round 2 : 15 selected.
AVG Round 2 : 4 selected.
Number of local servers : 4; 2 fold.
LSA Round 2 : 10 selected.
AVG Round 2 : 4 selected.
Number of local servers : 4; 3 fold.
LSA Round 2 : 15 selected.
AVG Round 2 : 4 selected.
Comparison of operation times of fed-GB-LSA (LSA) and fed-GB-Average (Avg)
operation times of fed-GB-LSA (LSA)
[16.62617785 32.36437637]
operation times of fed-GB-Average (Avg)
[28.05481213 56.75422299]


In [ ]:
# ray

In [24]:
from itertools import product
hh_list = [i for i in range(4)]
params = list(product(hh_list, numworkersseq))
print(params)

[(0, 2), (0, 4), (1, 2), (1, 4), (2, 2), (2, 4), (3, 2), (3, 4)]


In [ ]:
import ray
ray.init()

In [26]:
@ray.remote
def f(hh, numworkers):
    print(f'Number of local servers : {numworkers}; {hh} fold.')
    test = set([i for i in range(hh * 25, (hh + 1) * 25)])
    train = list(allsamples - test)
    test = setGenerator(test, samplesPerWorker, numworkers)
    predictors = np.zeros([len(yfine), samplesPerWorker * numworkers, numPredictors])
    response = np.zeros([len(yfine), samplesPerWorker * numworkers])
    x = []
    y = []
    for l in range(1, numworkers + 1):
        with open('tmp/yfdobj_' + str(l) + '_' + str(numworkers), 'rb') as file:
            yfdobj = pickle.load(file)
        with open('tmp/predictorLst_' + str(l) + '_' + str(numworkers), 'rb') as file:
            predictorLst = pickle.load(file)
        xfdobjTrainLst = []
        for i in range(numPredictors):
            predictors[:, (samplesPerWorker * (l - 1)): (samplesPerWorker * l), i] = eval_fd(yfine,
                                                                                             predictorLst[i])
            temp1 = smooth_basis(yfine, predictors[:, [i + samplesPerWorker * (l - 1) - 1 for i in train], i],
                                 basisobj).fd
            xfdobjTrainLst.append(temp1)
        x.append(xfdobjTrainLst)
        response[:, (samplesPerWorker * (l - 1)): (samplesPerWorker * l)] = eval_fd(yfine, yfdobj)
        responsefdobjTrain = smooth_basis(yfine,
                                          response[:, [i + samplesPerWorker * (l - 1) - 1 for i in train]],
                                          basisobj).fd
        y.append(responsefdobjTrain)

    yfdobjTest = smooth_basis(yfine, response[:, test], basisobj).fd
    responseTest = eval_fd(yfine, yfdobjTest)
    boost_control = 2
    step_length = 0.5
    lin = linmod(x[0][1], y[0], betaList)
    step_length = 0.5
    start = time.time()
    federatedFunctionalGradBoostLSA(x, y, betaList, boost_control, step_length)
    end = time.time()
    time1[numworkersseq.index(numworkers), hh] = end - start
    start = time.time()
    federatedFunctionalGradBoostAvg(x, y, betaList, boost_control, step_length)
    end = time.time()
    time2[numworkersseq.index(numworkers), hh] = end - start
    return (time1[numworkersseq.index(numworkers), hh], time2[numworkersseq.index(numworkers), hh])

results = [f.remote(h, n) for h, n in params]
print(ray.get(results)) # [0, 1, 4, 9]

(f pid=14672) Number of local servers : 2; 0 fold.
(f pid=43184) LSA Round 2 : 11 selected.
(f pid=25712) Number of local servers : 2; 2 fold. [repeated 7x across cluster]
(f pid=44332) LSA Round 2 : 8 selected. [repeated 4x across cluster]
(f pid=43184) AVG Round 2 : 0 selected.
(f pid=43356) LSA Round 2 : 1 selected. [repeated 3x across cluster]
(f pid=14328) AVG Round 2 : 4 selected. [repeated 4x across cluster]
[(14.200671911239624, 24.632554531097412), (28.565675020217896, 60.843722343444824), (13.84568738937378, 23.259998083114624), (28.811443328857422, 61.1379611492157), (14.641746759414673, 22.503676176071167), (30.679824829101562, 63.8151068687439), (14.065617799758911, 23.426902532577515), (28.66248345375061, 59.91966509819031)]
(f pid=43356) AVG Round 2 : 4 selected.


In [25]:
for param in params:
    print(param)

(0, 2)
(0, 4)
(1, 2)
(1, 4)
(2, 2)
(2, 4)
(3, 2)
(3, 4)
